In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import sys, os
sys.path.insert(0, os.path.abspath('./model_utils'))
from window_generator import WindowGenerator

In [2]:
n =  13
rng = np.random.default_rng(seed=0)
df = pd.DataFrame(np.around(rng.random((n, 4)), 1), columns=['a','b','c','y'])
# df.iloc[1:2,:] = np.nan
# df.iloc[7:9,:] = np.nan
# print(df)

In [3]:
# Window generator
OUT_STEPS = 2  
input_width = 2   # Take two rows (or time steps) of all columns as input
label_width = OUT_STEPS  # Size of the prediction (output)
shift = OUT_STEPS  # Time (or rows) offset between input and output
batch_size = 1
label_column='y'
# label_index = df.columns.get_loc(label_column)
label_index = None
# Single step window
my_window = WindowGenerator(
                input_width=input_width,
                label_width=label_width,
                shift=shift,
                batch_size=batch_size,
                train_df=df, val_df=df, test_df=df,
                label_columns=[label_column],
                use_label_columns=True, shuffle=False
                )

https://www.tensorflow.org/api_docs/python/tf/keras/Model  
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Average   


In [4]:
class MyBaseline(tf.keras.Model):
    """
    Calculates the mean value. Returns it as the target.
    https://www.tensorflow.org/tutorials/structured_data/time_series#baselines
    """
    def __init__(self, label_index=None):
        """ 
        label_index: an int with the number of the column to be evaluated
        OUT_STEPS: Number of times the last value (label_width) is repeated
        """
        super().__init__()
        self.label_index = label_index
        # self.average = tf.keras.layers.Average()

    def call(self, inputs): # inputs.shape: (None, 2, 4) <class 'tensorflow.python.framework.ops.Tensor'>
        """
        Calls the model on new inputs.
        Returns the outputs as tensors.
        https://www.tensorflow.org/api_docs/python/tf/keras/Model#call
        """

        out_steps = inputs.shape[1]
        averages = tf.reduce_mean(inputs, axis=1)
        averages = averages[:,tf.newaxis,:]
        result = tf.tile(averages, [1, out_steps, 1])

        if self.label_index is None:
            print(f'result: {result}')
            print(f'result.shape: {result.shape}')
            return result
        result = result[:, :, self.label_index]
        return result[:, :, tf.newaxis]




In [5]:
# Baseline from WindowGenerator
baseline_model = MyBaseline()
# baseline_multi = RepeatBaseline(label_index=label_index)
baseline_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()]) # MAE
# Evaluation from WindowGenerator
dataset = my_window.val
evaluation = baseline_model.evaluate(dataset, verbose=0)
print(evaluation)

result: Tensor("my_baseline/Tile:0", shape=(None, 2, 4), dtype=float32)
result.shape: (None, 2, 4)
[0.1264375001192093, 0.29499998688697815]


In [6]:
for batch in dataset.take(3):
    inputs, targets = batch
    print(f'input: {inputs} |\ttarget: {targets}')
    # print(f'target: {targets}')
print(f'input.shape: {inputs.shape}')
print(f'target.shape: {targets.shape}')

input: [[[0.6 0.3 0.  0. ]
  [0.8 0.9 0.6 0.7]]] |	target: [[[0. ]
  [0.2]]]
input: [[[0.8 0.9 0.6 0.7]
  [0.5 0.9 0.8 0. ]]] |	target: [[[0.2]
  [0.4]]]
input: [[[0.5 0.9 0.8 0. ]
  [0.9 0.  0.7 0.2]]] |	target: [[[0.4]
  [0.6]]]
input.shape: (1, 2, 4)
target.shape: (1, 2, 1)


In [7]:
def naive_mae_multi(model , dataset_, label_index_):
    targets = []
    for batch_ in dataset_:
        _, target_ = batch_
        targets.append(target_.numpy())
    targets = np.array(targets)   
    predictions = model.predict(dataset_, verbose = 0)
    targets = targets.reshape([targets.shape[0], targets.shape[2], targets.shape[1]])
    mae = np.abs(predictions - targets).mean()
    return mae, predictions
    
mae, predictions = naive_mae_multi(baseline_model, dataset, label_index)
print(f'The MAE: {mae}')
print(predictions)

result: Tensor("my_baseline/Tile:0", shape=(None, 2, 4), dtype=float32)
result.shape: (None, 2, 4)
The MAE: 0.29499998688697815
[[[0.70000005 0.6        0.3        0.35      ]
  [0.70000005 0.6        0.3        0.35      ]]

 [[0.65       0.9        0.70000005 0.35      ]
  [0.65       0.9        0.70000005 0.35      ]]

 [[0.7        0.45       0.75       0.1       ]
  [0.7        0.45       0.75       0.1       ]]

 [[0.9        0.25       0.5        0.3       ]
  [0.9        0.25       0.5        0.3       ]]

 [[0.45       0.3        0.5        0.5       ]
  [0.45       0.3        0.5        0.5       ]]

 [[0.3        0.25       0.85       0.8       ]
  [0.3        0.25       0.85       0.8       ]]

 [[0.65       0.55       0.85       0.7       ]
  [0.65       0.55       0.85       0.7       ]]

 [[0.4        0.7        0.6        0.35000002]
  [0.4        0.7        0.6        0.35000002]]

 [[0.3        0.79999995 0.7        0.35000002]
  [0.3        0.79999995 0.7        0.35

In [8]:
# >> predictions 
# >> array([[[0.7, 0.6, 0.3, 0.35],
    #         [0.7, 0.6, 0.3, 0.35]],

    #        [[0.65, 0.9, 0.7, 0.35],
    #         [0.65, 0.9, 0.7, 0.35]],

    # ...  More data here           ]]